In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
from tensorflow.keras.models import load_model
import numpy as np
np.random.seed(1337)  # for reproducibility
import os
import argparse
from google.colab.patches import cv2_imshow 
#from keras.utils.visualize_util import plot
from tensorflow.keras.preprocessing.image import img_to_array
import tensorflow as tf
import cv2
from tensorflow.keras import Input
import getpass as gp
import sys
from tensorflow.keras.models import Model
import random
random.seed(1337)
import matplotlib.pyplot as plt
import joblib
from skimage import io, filters
from skimage.feature import hog, ORB, CENSURE, corner_peaks, corner_harris, BRIEF
from skimage.transform import resize
import warnings
warnings.filterwarnings('ignore')

In [3]:
def processImg(that_img , img_size=None):
  DataSet = []
  LabelSet = []
  lengthV = []
  resList = []
  boolList = []
  pos = 0
  ind = 0
  useList = [True, True, True, True]
  #initialize feature detectors/extractors
  #Censure extractor
  detector = CENSURE()
  #ORB extractor
  detector2 = ORB(n_keypoints=50)

  featureVector = []
  img = that_img
  #get histogram for grayscale value intensity
  hist = np.histogram(img, bins=256)
  #resize image
  img = resize(img, (400,400))
  #extract features but do not yet add them to feature vector
  detector2.detect_and_extract(img)
  #extract HOG features, add them to featurevector
  a = fd = hog(img, orientations=9, pixels_per_cell=(32, 32),
          cells_per_block=(1,1), visualize=False)
  #add histogramm to featurevector
  for h in hist:
      fd = np.append(fd, h)
  #if corresponding boolean in uselist is true add features to featureVector --> Feature selection happens here
  if(useList[0]):                            
      detector.detect(img)
      fd = np.append(fd, [np.array(detector.keypoints).flatten()])
  if(useList[1]):
      fd = np.append(fd, detector2.keypoints)
  if(useList[2]):
      fd = np.append(fd, edgeExtract(img, 100))
  if(useList[3]):
      corners =  corner_peaks(corner_harris(img),min_distance=1)
      fd = np.append(fd, corners)
  #get length of featurevector for later operations
  lengthV.append(len(fd))
  #add featureVector list to dataset that is fed into svm
  DataSet.append(fd)
  max = 17373
  lengthV = []
  DataSet2 = []
  #pad dataset with zeroes so that all featurevectors have the same length --> important for svm
  for d in DataSet:
      d = np.pad(d, (0, max - len(d)), 'constant')
      DataSet2.append(d)
      lengthV.append(len(d))
  DataSet = DataSet2
  return DataSet

In [4]:
#extract edge histogramm, bins number = 100    
def edgeExtract(img, bins):
    retVal = []
    #apply vertical and horizontal sobel filters to get two histogramms, once of vertical and once of horizontal edges
    #vertical
    fs = filters.sobel_v(img)
    #horizontal
    angs = filters.sobel_h(img)
    #compute histograms
    lhist = np.histogram(fs,bins,normed=True,range=(0,1))
    ahist = np.histogram(angs, bins,normed=True,range=(-180,180))
    #fuse histograms into one list
    retVal.extend(lhist[0].tolist())
    retVal.extend(ahist[0].tolist())
    return retVal

In [ ]:
test_data_dir = '/content/drive/My Drive/Đồ án tốt nghiệp - Nghĩa, Duy/Dataset/Starver_split_8_2/test/copies_crop/'
classes=["stamp","no_stamp"]


class_s_test=os.listdir(test_data_dir+classes[0])
class_ns_test=os.listdir(test_data_dir+classes[1])
img_size = (128 , 128)
print("Number of forgery images :", len(class_s_test))
print("Number of original images :", len(class_ns_test))

Number of forgery images : 19
Number of original images : 79


In [ ]:
model = load_model("/content/drive/MyDrive/Đồ án tốt nghiệp - Nghĩa, Duy/Model/Date 29-10/classification_1_model.h5")

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 32)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 128)               2

In [ ]:
# test stamp img
demo = random.sample(class_s_test,3)
for name in demo:
  # print(name)
  stamp_img = cv2.imread(test_data_dir+"/"+classes[0]+"/"+name)
  img = cv2.resize(stamp_img,(64,64))
  img = np.reshape(img,[1,64,64,3])
  prediction = (model.predict(img) <0.6).astype("int32")
  cv2_imshow(stamp_img)
  if prediction == 0:
    result="Stamp"
  else:
    result="Not Stamp"
  print("Predict: "+result+"    Actual: Stamp")

# test not stamp img
demo = random.sample(class_ns_test,3)
for name in demo:
  # print(name)
  not_stamp_img = cv2.imread(test_data_dir+"/"+classes[1]+"/"+name)
  img = cv2.resize(not_stamp_img,(64,64))
  img = np.reshape(img,[1,64,64,3])
  prediction = (model.predict(img) < 0.6).astype("int32")
  cv2_imshow(not_stamp_img)
  if prediction == 0:
    result="Stamp"
  else:
    result="Not Stamp"
  print("Predict: "+result+"    Actual: Not Stamp")

Output hidden; open in https://colab.research.google.com to view.

In [5]:
test_data_dir = '/content/drive/My Drive/Đồ án tốt nghiệp - Nghĩa, Duy/Dataset/Starver_split_8_2/test/second_model_data/'
classes=["copies","genuine"]


forg_test=os.listdir(test_data_dir+classes[0]+"/stamp/")
org_test=os.listdir(test_data_dir+classes[1]+"/stamp/")
img_size = (128 , 128)
print("Number of forgery images :", len(forg_test))
print("Number of original images :", len(org_test))

Number of forgery images : 19
Number of original images : 101


In [6]:
model_2=joblib.load("/content/drive/MyDrive/Đồ án tốt nghiệp - Nghĩa, Duy/Model/Date 26-11/classification_2_modelSVC, rbf.pk1")

In [7]:
print(model_2)

SVC(C=10.0, gamma=1e-09, probability=True)


In [14]:

# test fake img
demo = random.sample(forg_test,3)
for name in demo:
  # print(name)
  forg_img = cv2.imread(test_data_dir+"/"+classes[0]+"/stamp/"+name)
  cv2_imshow(forg_img)
  forg_img = cv2.cvtColor(forg_img, cv2.COLOR_BGR2GRAY)
  forg_img_processed = processImg(forg_img)
  result = model_2.predict(forg_img_processed)
  # print(result)
  # result = (model_2.predict(forg_img_processed) < 0.5).astype("int32")
  # print(result)
  if result == 0:
    result="Fake"
  else:
    result="Real"
  print("Predict: "+result+"    Actual: Fake")

# test real img
demo = random.sample(org_test,3)
for name in demo:
  # print(name)
  org_img = cv2.imread(test_data_dir+"/"+classes[1]+"/stamp/"+name)
  cv2_imshow(org_img)
  org_img = cv2.cvtColor(org_img, cv2.COLOR_BGR2GRAY)
  org_img_processed = processImg(org_img)
  result = model_2.predict(org_img_processed)
  # print(result)
  if result == 0:
    result="Fake"
  else:
    result="Real"
  print("Predict: "+result+"    Actual: Real")

Output hidden; open in https://colab.research.google.com to view.